In [1]:
import numpy as np
import pandas as pd
import nltk
import sklearn
import operator
import requests
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import re
from collections import Counter
from nltk.util import ngrams
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Surface\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Surface\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Surface\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#we start by reading each of the files provided, and saving them in the relevant variables
#note: I've added a certain text encoding as I experienced errors when useing the default...


#first read train posivie and negative files, and add each to a matrix
path_train_pos='./IMDb/train/imdb_train_pos.txt'
dataset_train_pos=open(path_train_pos,encoding='utf8').readlines()

path_train_neg='./IMDb/train/imdb_train_neg.txt'
dataset_train_neg=open(path_train_neg,encoding='utf8').readlines()

#now read test positive and negative files, and add each to a matrix
path_test_pos='./IMDb/test/imdb_test_pos.txt'
dataset_test_pos=open(path_test_pos,encoding='utf8').readlines()

path_test_neg='./IMDb/test/imdb_test_neg.txt'
dataset_test_neg=open(path_test_neg,encoding='utf8').readlines()

#now read develop positive and negative files, and add each to a matrix
path_dev_pos='./IMDb/dev/imdb_dev_pos.txt'
dataset_dev_pos=open(path_dev_pos,encoding='utf8').readlines()

path_dev_neg='./IMDb/dev/imdb_dev_neg.txt'
dataset_dev_neg=open(path_dev_neg,encoding='utf8').readlines()

#dataset_file2=pd.read_csv('C:/Users/Surface/Documents/Applied Machine Learning/datasets_coursework1/IMDb/train/imdb_train_neg.txt',encoding='utf8')
#print(dataset_file2.head(5))


In [3]:
#print the first 3 entries of positive train, file name: imdb_train_pos to make sure file is opened successfully...
print ("Positive reviews from the training dataset:\n")
for i in dataset_train_pos[:1]:
  print (i)
print ("\n   ------------------------\n")  



Positive reviews from the training dataset:

For fans of Chris Farley, this is probably his best film. David Spade plays the perfect cynical, sarcastic yin to Farley's "Baby Huey" yang. Farley achieves strokes of comic genius in his monologues, like the "Let's say you're driving along the road with your family..." bit, the "Jo-Jo the Idiot Circus Boy with a pretty new pet, (his possible sale)" speech, or the "Glue-sniffing Guarantee fairy" brake pad sale. The sappy moments in the film contrast sharply with Farley and Spade's shenanigans. Even after many viewings, it's still fun to see Farley pour everything he had into the role. "Richard, what's HAPPENING to me?!?!"


   ------------------------



In [4]:
#now we join pos and neg of each set into 1 set:

new_train_set=[]
for pos_review in dataset_train_pos:
  new_train_set.append((pos_review,1))
for neg_review in dataset_train_neg:
  new_train_set.append((neg_review,0))


new_test_set=[]
for pos_review in dataset_test_pos:
  new_test_set.append((pos_review,1))
for neg_review in dataset_test_neg:
  new_test_set.append((neg_review,0))


new_dev_set=[]
for pos_review in dataset_dev_pos:
  new_dev_set.append((pos_review,1))
for neg_review in dataset_dev_neg:
  new_dev_set.append((neg_review,0))

In [5]:
#here we just print the first entry for each set to make sure the mering was successful
print ("TRAINING SET")
print ("Size training set: "+str(len(new_train_set)))
for example in new_train_set[:1]:
    print (example)
print ("    \n-------\n")
print ("TEST SET")
print ("Size development set: "+str(len(new_dev_set)))
for example in new_dev_set[:1]:
  print (example)
print ("    \n-------\n")
print ("DEVELOPMENT SET")
print ("Size test set: "+str(len(new_test_set)))
for example in new_test_set[:1]:
  print (example)

TRAINING SET
Size training set: 15000
('For fans of Chris Farley, this is probably his best film. David Spade plays the perfect cynical, sarcastic yin to Farley\'s "Baby Huey" yang. Farley achieves strokes of comic genius in his monologues, like the "Let\'s say you\'re driving along the road with your family..." bit, the "Jo-Jo the Idiot Circus Boy with a pretty new pet, (his possible sale)" speech, or the "Glue-sniffing Guarantee fairy" brake pad sale. The sappy moments in the film contrast sharply with Farley and Spade\'s shenanigans. Even after many viewings, it\'s still fun to see Farley pour everything he had into the role. "Richard, what\'s HAPPENING to me?!?!"\n', 1)
    
-------

TEST SET
Size development set: 5000
('After 10 viewings in 20 years I too think this was the Crazy Gang\'s best effort on film, with more cohesion in the plot than their next best, "Alf\'s Button Afloat". They were indeed a crazy trio of double acts thrown together mainly on stage, sometimes in front o

In [6]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    list_tokens=[]
    for token in tokens:
        list_tokens.append(lemmatizer.lemmatize(token).lower())
    # the following part shall the tokens in and return POS tuple with word type tag, 
    # we shall then be taking adjectives and verbs only
    test = nltk.pos_tag(list_tokens)
    final_tokens = [a[0] for a in test if (a[1] == 'JJ' or a[1] == 'VERB')]
    return final_tokens
 
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # clean doc
    tokens = clean_doc(filename)
    # update counts
    vocab.update(tokens)
 
 
# define vocab
vocab = Counter()
# add all docs to vocab
for instance in ([i[0] for i in new_train_set]):
    add_doc_to_vocab(instance, vocab)

# print the size of the vocab
print(len(vocab))
# print the top 50 words in the vocab
print(vocab.most_common(50))


29803
[('good', 8100), ('great', 5386), ('bad', 5340), ('many', 4127), ('much', 4088), ('little', 3368), ('first', 2927), ('real', 2718), ('old', 2334), ('new', 2268), ('funny', 2089), ('big', 2028), ('young', 1951), ('whole', 1748), ('br', 1745), ('original', 1706), ('ive', 1683), ('last', 1680), ('u', 1543), ('main', 1414), ('different', 1349), ('sure', 1310), ('american', 1289), ('special', 1286), ('dont', 1275), ('true', 1239), ('black', 1232), ('hard', 1208), ('second', 1169), ('high', 1159), ('short', 1151), ('poor', 1146), ('cant', 1104), ('classic', 1076), ('give', 1066), ('excellent', 1039), ('beautiful', 1029), ('right', 1028), ('full', 1018), ('human', 978), ('nice', 972), ('stupid', 965), ('interesting', 960), ('dead', 941), ('terrible', 925), ('wrong', 920), ('im', 918), ('enough', 900), ('long', 890), ('small', 888)]


In [7]:
#this function shall take care tokenizing the text, normalizing and cleaning as per clean_doc function, and vectorinzation as well.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=clean_doc,ngram_range=(1,2),max_features=500)
matrix = vectorizer.fit_transform([i[0] for i in new_train_set])
a = matrix.toarray()

In [8]:
#now we train our model, this shall take some time.
import time
start = time.time()
X_train=a
Y_train=[i[1] for i in new_train_set]
svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf.fit(np.asarray(X_train),np.asarray(Y_train))
end = time.time()

print("Time to train the model is: %f seconds"%(float(end)- float(start)))


Time to train the model is: 83.504465 seconds


In [9]:
#here we shall start checking the performance or our model using the development set
#we start with the trained model vectorizer that uses most common 500 words and ngrams = 2

X_dev = vectorizer.transform([i[0] for i in new_dev_set]).toarray()
predictions = svm_clf.predict(X_dev)
print(sklearn.metrics.classification_report(predictions,[i[1] for i in new_dev_set]))

              precision    recall  f1-score   support

           0       0.76      0.79      0.77      2394
           1       0.80      0.77      0.78      2606

    accuracy                           0.78      5000
   macro avg       0.78      0.78      0.78      5000
weighted avg       0.78      0.78      0.78      5000



In [10]:
#we notice that the accuracy is not bad, but it could be improved. 
#we will use the development set to try some fine tuning, here we will consider 2000 words, and ngrams = 3


vectorizer2 = TfidfVectorizer(tokenizer=clean_doc,ngram_range=(1,3),max_features=2000)
matrix = vectorizer2.fit_transform(np.asarray([i[0] for i in new_train_set]))
X_train = matrix.toarray()
Y_train=[i[1] for i in new_train_set]

svm_clf2=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf2.fit(np.asarray(X_train),np.asarray(Y_train))

X_dev = vectorizer2.transform([i[0] for i in new_dev_set]).toarray()
predictions = svm_clf2.predict(X_dev)
print(sklearn.metrics.classification_report(predictions,[i[1] for i in new_dev_set]))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79      2370
           1       0.82      0.79      0.80      2630

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



In [11]:
#here we shall try to reduce the dimentionality of our model, by selecting the features with highest score
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

vectorizer_f = TfidfVectorizer(tokenizer=clean_doc,ngram_range=(1,2),max_features=5000)
X_train = vectorizer_f.fit_transform([i[0] for i in new_train_set]).toarray()
Y_train = np.asarray([i[1] for i in new_train_set])

X_feature_best = SelectKBest(chi2, k=500).fit(X_train, Y_train)
X_train_feature_best = X_feature_best.transform(X_train)

X_test = vectorizer_f.transform([i[0] for i in new_test_set]).toarray()
X_test_feature_best =  X_feature_best.transform(X_test)

Y_test = np.asarray([i[1] for i in new_test_set])

In [12]:
#finally we train our model and predict the results using the test dataset
svm_clf_3=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf_3.fit(np.asarray(X_train_feature_best),Y_train)
predictions_feature_best = svm_clf_3.predict(X_test_feature_best)
print(sklearn.metrics.classification_report(predictions_feature_best,Y_test))



              precision    recall  f1-score   support

           0       0.75      0.83      0.79      2284
           1       0.84      0.77      0.81      2716

    accuracy                           0.80      5000
   macro avg       0.80      0.80      0.80      5000
weighted avg       0.80      0.80      0.80      5000



In [13]:
#we are going to measure precision, recall, F1 score and Accuracy here
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
Y_test_gold = Y_test
Y_text_predictions = predictions_feature_best
precision=precision_score(Y_test_gold, Y_text_predictions, average='macro')
recall=recall_score(Y_test_gold, Y_text_predictions, average='macro')
f1=f1_score(Y_test_gold, Y_text_predictions, average='macro')
accuracy=accuracy_score(Y_test_gold, Y_text_predictions)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.799
Recall: 0.797
F1-Score: 0.797
Accuracy: 0.797


In [14]:
#we could check the confusion matrix to verify tn, fp, fn, tp
from sklearn.metrics import confusion_matrix
print (confusion_matrix(Y_test_gold, Y_text_predictions))

[[1885  616]
 [ 399 2100]]
